### Check the downloaded file and reformat the data

In [1]:
from tqdm.auto import tqdm
import pandas as pd

In [2]:
covid_df = pd.read_csv('../data/2020-05-31_18-12.csv', sep=';')

In [3]:
covid_df.head()

,Дата,Регион,Заражений,Выздоровлений,Смертей,Смертей за день,Заражений за день,Выздоровлений за день
0,02.03.2020,Московская обл.,1,0,0,0,1,0
1,03.03.2020,Московская обл.,1,0,0,0,0,0
2,04.03.2020,Московская обл.,1,0,0,0,0,0
3,05.03.2020,Московская обл.,1,0,0,0,0,0
4,06.03.2020,Москва,5,0,0,0,5,0


### Check the list of Russian regions

In [4]:
covid_df['Регион'].unique()

array(['Московская обл.', 'Москва', 'Нижегородская обл.', 'Липецкая обл.',
       'Санкт-Петербург', 'Белгородская обл.', 'Калининградская обл.',
       'Адыгея', 'Алтайский край', 'Амурская обл.', 'Архангельская обл.',
       'Астраханская обл.', 'Башкортостан', 'Брянская обл.', 'Бурятия',
       'Владимирская обл.', 'Волгоградская обл.', 'Вологодская обл.',
       'Воронежская обл.', 'Дагестан', 'Еврейская АО',
       'Забайкальский край', 'Ивановская обл.', 'Ингушетия',
       'Иркутская обл.', 'Кабардино-Балкария', 'Калмыкия',
       'Калужская обл.', 'Камчатский край', 'Карачаево-Черкессия',
       'Карелия', 'Кемеровская обл.', 'Кировская обл.', 'Коми',
       'Костромская обл.', 'Краснодарский край', 'Красноярский край',
       'Крым', 'Курганская обл.', 'Курская обл.', 'Ленинградская обл.',
       'Магаданская обл.', 'Марий Эл', 'Мордовия', 'Мурманская обл.',
       'Новгородская обл.', 'Новосибирская обл.', 'Омская обл.',
       'Оренбургская обл.', 'Орловская обл.', 'Пензенск

In [5]:
regions = covid_df['Регион'].unique()

In [6]:
covid_df['Регион'].unique().shape

(85,)

This is a valid number

### We will make several separate files, where each will contain data for only one day, but for all regions at once

In [7]:
unique_dates = covid_df['Дата'].unique()

In [8]:
for date_index in tqdm(range(len(unique_dates))):
    each_date = unique_dates[date_index]
    this_date_df = covid_df[covid_df['Дата'] == each_date]
    
    # infections
    this_date_df_infections = this_date_df[['Регион', 'Заражений за день']]
    this_date_df_infections = this_date_df_infections.rename(columns={'Регион': 'region', 'Заражений за день': 'infections'})
    this_date_df_infections = this_date_df_infections.set_index('region')
    
    # recoveries
    this_date_df_recoveries = this_date_df[['Регион', 'Выздоровлений за день']]
    this_date_df_recoveries = this_date_df_recoveries.rename(columns={'Регион': 'region', 'Выздоровлений за день': 'recoveries'})
    this_date_df_recoveries = this_date_df_recoveries.set_index('region')
    
    # deaths
    this_date_df_deaths = this_date_df[['Регион', 'Смертей за день']]
    this_date_df_deaths = this_date_df_deaths.rename(columns={'Регион': 'region', 'Смертей за день': 'deaths'})
    this_date_df_deaths = this_date_df_deaths.set_index('region')
    
    # new_df
    new_this_date_df = pd.DataFrame(columns=['region', 'infections', 'recoveries', 'deaths'])
    new_this_date_df['region'] = regions
    new_this_date_df = new_this_date_df.set_index('region')
    
    # update
    new_this_date_df.update(this_date_df_infections)
    new_this_date_df.update(this_date_df_recoveries)
    new_this_date_df.update(this_date_df_deaths)
    
    # reset index
    new_this_date_df.reset_index(inplace=True)
    
    # fill na with zeros
    new_this_date_df = new_this_date_df.fillna(0)
    
    new_this_date_df.to_csv('../data/' + each_date + '.csv', sep=';', index=False)